<a href="https://colab.research.google.com/github/igorrendulic/ai_heckathon_tw_25/blob/master/ai_hackathon_tw_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [7]:
!pip install pytube
!pip install pytubefix
!pip install ffmpeg-python
!pip install transformers
!pip install pydub
!pip install openai-whisper

In [9]:
import torch
import ffmpeg
from pytubefix import YouTube
import os
from typing import List
import sys
from pydub import AudioSegment
import numpy as np
from tqdm.auto import tqdm
import whisper
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

In [10]:
yt = YouTube('https://www.youtube.com/watch?v=l8L9Z2vmMTQ', 'WEB')

In [ ]:
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download("sample_data")

'/content/sample_data/The Science of the Matrix with Laurence Fishburne.mp4'

In [11]:
def list_mp4s(folder:str) -> List[str]:
    mp4s = []
    for file in os.listdir(folder):
        if file.endswith(".mp4"):
            mp4s.append(file)
    return mp4s

In [12]:
def decode_audio(in_filename, **input_kwargs):
    try:
        out, err = (ffmpeg
            .input(in_filename, **input_kwargs)
            .output('-', format='s16le', acodec='pcm_s16le', ac=1, ar='16k')
            .overwrite_output()
            .run(capture_stdout=True, capture_stderr=True)
        )
    except ffmpeg.Error as e:
        print(e.stderr, file=sys.stderr)

    return out


In [60]:
mp4s = list_mp4s("sample_data")
mp4_path = None
if len(mp4s) > 0:
  mp4_path = os.path.join("sample_data", mp4s[0])

audio_data = None
if mp4_path is None:
  print("No mp4s found")
else:
  audio_data = decode_audio(mp4_path)


In [ ]:
with open("sample_data/audio.pcm", "wb") as f:
    f.write(audio_data)

In [ ]:
aac = AudioSegment.from_file("sample_data/audio.pcm", "aac", sample_width=2, frame_rate=16000, channels=1)
aac.export('sample_data/audio.mp3', format='mp3', codec='mp3')

<_io.BufferedRandom name='sample_data/audio.mp3'>

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [ ]:
whisper_model_size = "medium.en"
model = whisper.load_model(whisper_model_size)

100%|█████████████████████████████████████| 1.42G/1.42G [01:13<00:00, 20.7MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
tr_result = model.transcribe("sample_data/audio.mp3")

In [15]:
emb_index = []
segments = tr_result["segments"]
for segment in segments:
  emb_index.append({
      "start": segment["start"],
      "end": segment["end"],
      "text": segment["text"]
  })

NameError: name 'tr_result' is not defined

In [ ]:
import json
with open("sample_data/segments.json", "w") as f:
  json.dump(emb_index, f)

In [ ]:
with open("sample_data/text.json", "w") as f:
  f.write(tr_result["text"])

In [30]:
with open("sample_data/segments.json", "r") as f:
   segments = json.load(f)

In [26]:
embedding_model_id = "intfloat/e5-large-v2"

In [27]:
emb_tokenizer = AutoTokenizer.from_pretrained(embedding_model_id)
emb_model = AutoModel.from_pretrained(embedding_model_id)
emb_model.eval()
emb_model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

In [31]:
print(f"max token length: {emb_tokenizer.model_max_length}")

max token length: 512


In [38]:
# assembline chunks and expending start, end time (assuming 4 words is a token, max-length - 50 just to be sure)
import warnings

chunked_segments = []
all_chunked_segments = []
# segment_size = emb_tokenizer.model_max_length - 50
# experiment with the size of the chunk (effect the end clip size)
init_segment_size = int(emb_tokenizer.model_max_length / 3)
segment_size = init_segment_size

print(f"Segment size: {init_segment_size}")

for i, segment in tqdm(enumerate(segments), desc="Chunking segments"):
  start = segment["start"]
  end = segment["end"]
  text = segment["text"].strip()
  num_words_approx = len(text.split(" "))

  # if num_words_approx > segment_size:
  #   warnings.warn(f"Segment too long ({num_words_approx} words), consider adding code for splitting. Hopefully no need for it.")

  if num_words_approx < segment_size:
    chunked_segments.append({"start": start, "end": end, "text": text})
    segment_size -= num_words_approx
  else:
    all_chunked_segments.append(chunked_segments)
    first_chunk = chunked_segments[0]
    last_chunk = chunked_segments[-1]
    print(f"num segments joined: {len(chunked_segments)}, start: {first_chunk['start']}, end: {last_chunk['end']}")
    # print(f"chunk assembled size: {len(chunked_segments)}, start: {chunked_segments["start"]}, end: {chunked_segments["end"]}")
    # restart the process for new chunks
    chunked_segments = []
    segment_size = init_segment_size
    chunked_segments.append({"start": start, "end": end, "text": text})
    all_chunked_segments.append(chunked_segments)

# if anything left
if len(chunked_segments) > 0:
  all_chunked_segments.append(chunked_segments)

Segment size: 170


Chunking segments: 0it [00:00, ?it/s]

num segments joined: 29, start: 0.0, end: 85.0
num segments joined: 20, start: 85.0, end: 133.0
num segments joined: 25, start: 133.0, end: 179.0
num segments joined: 14, start: 179.0, end: 226.0
num segments joined: 53, start: 226.0, end: 293.0
num segments joined: 27, start: 293.0, end: 346.48
num segments joined: 27, start: 346.48, end: 387.76000000000005
num segments joined: 20, start: 387.76000000000005, end: 436.36
num segments joined: 35, start: 436.36, end: 494.76
num segments joined: 43, start: 494.76, end: 551.18
num segments joined: 35, start: 551.18, end: 611.98
num segments joined: 51, start: 611.98, end: 686.98
num segments joined: 47, start: 687.58, end: 746.78
num segments joined: 47, start: 746.78, end: 801.78
num segments joined: 39, start: 801.78, end: 861.8199999999999
num segments joined: 15, start: 861.8199999999999, end: 919.72
num segments joined: 32, start: 919.8000000000001, end: 985.32
num segments joined: 30, start: 985.32, end: 1039.4399999999998
num segmen

In [39]:
len(all_chunked_segments), len(segments)

(123, 1958)

In [40]:
def mean_pooling(outputs, attention_mask):
  # do mean pooling
  token_embeddings = outputs.last_hidden_state
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
  pooled = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
  return pooled


In [41]:
# combine the chunks into a single text and create emebddings
# record the start, end time of the each chunk
emb_index = []
embeddings = []

processed_chunks = set()

for i, chunk in tqdm(enumerate(all_chunked_segments), desc="embedding chunked segments", total=len(all_chunked_segments)):
  text = ""
  start = 0.0
  end = 0.0

  print(f"Processing chunk {i}: {chunk}")

  for js, segment in enumerate(chunk):
    if js == 0:
      start = segment["start"]
    if js == len(chunk) - 1:
      end = segment["end"]
    text += segment["text"]

  text = text.strip()
  # print(f"emb index start: {start}, end: {end}")

  if start in processed_chunks:
    print(f"this was already processed: {start}")
    continue

  emb_index.append({
      "start": start,
      "end": end,
      "text": text
  })

  processed_chunks.add(start)

  inputs = emb_tokenizer(text, padding=True, truncation=True, return_tensors="pt")
  inputs = inputs.to(emb_model.device)
  with torch.no_grad():
    outputs = emb_model(**inputs)

  # do mean pooling
  pooled = mean_pooling(outputs, inputs['attention_mask'])

  embeddings.append(pooled.cpu().numpy())




embedding chunked segments:   0%|          | 0/123 [00:00<?, ?it/s]

Processing chunk 0: [{'start': 0.0, 'end': 2.0, 'text': "Don't try to out-deep voice me."}, {'start': 2.0, 'end': 3.0, 'text': 'Okay.'}, {'start': 3.0, 'end': 4.0, 'text': "I won't."}, {'start': 4.0, 'end': 7.0, 'text': 'So, how deep can you take your voice?'}, {'start': 7.0, 'end': 9.0, 'text': 'Probably right about in here.'}, {'start': 9.0, 'end': 11.0, 'text': 'Like, below audio frequency levels?'}, {'start': 11.0, 'end': 13.0, 'text': "I don't know. It's that deep."}, {'start': 13.0, 'end': 15.0, 'text': "Let me say, I haven't done this in a while."}, {'start': 15.0, 'end': 17.0, 'text': "I'm Morpheus."}, {'start': 17.0, 'end': 18.0, 'text': 'No.'}, {'start': 19.0, 'end': 21.0, 'text': "I'm Morpheus."}, {'start': 22.0, 'end': 24.0, 'text': "Why can't I be Morpheus?"}, {'start': 24.0, 'end': 26.0, 'text': "Because you're not."}, {'start': 26.0, 'end': 29.0, 'text': 'Because I am.'}, {'start': 57.0, 'end': 59.0, 'text': 'Lawrence Fishburne. Lawrence!'}, {'start': 59.0, 'end': 61.0, 

In [66]:
# sanity check
assert len(emb_index) == len(embeddings)
print(f"Number of embeddings: {len(embeddings)}, emb_index: {len(emb_index)}")

Number of embeddings: 62, emb_index: 62


In [67]:
# save embeddings index and embeddings somewhere
import json
with open("sample_data/emb_index.json", "w") as f:
  json.dump(emb_index, f)

embeddings_stacked = np.vstack(embeddings)
np.save("sample_data/embeddings.npy", embeddings_stacked)

In [68]:
emb_index

[{'start': 0.0,
  'end': 85.0,
  'text': "Don't try to out-deep voice me.Okay.I won't.So, how deep can you take your voice?Probably right about in here.Like, below audio frequency levels?I don't know. It's that deep.Let me say, I haven't done this in a while.I'm Morpheus.No.I'm Morpheus.Why can't I be Morpheus?Because you're not.Because I am.Lawrence Fishburne. Lawrence!Neil.How are you, man?Welcome to Third Talk!Thank you. This is so great. Thank you for having me, man.Oh my God.I'm so excited to be here.This is your first time back at the Hayden Planetarium?I haven't been to the Hayden Planetarium since I was a boy, probably.Oh my gosh.Walked by many times.Yeah, because when I was a boy, I came by and was like, I'm doing this for life.Yes, exactly.I wanted to live here too, but I wasn't that good in science.I liked it, but I wasn't that good."},
 {'start': 85.0,
  'end': 133.0,
  'text': "Wait a minute. You told me your mama taught science.My mama was a science teacher, yeah.Okay, so

In [47]:
# load embeddings
emb_index = json.load(open("sample_data/emb_index.json", "r"))
embeddings = np.load("sample_data/embeddings.npy")
embeddings = torch.from_numpy(embeddings).to(device)

In [69]:
def search(query, k=5):
  inputs = emb_tokenizer(query, padding=True, truncation=True, return_tensors="pt")
  inputs = inputs.to(emb_model.device)
  with torch.no_grad():
    outputs = emb_model(**inputs)

  pooled = mean_pooling(outputs, inputs['attention_mask'])

  q = F.normalize(pooled, p=2, dim=1)
  similarity = F.cosine_similarity(q, embeddings, dim=1)
  values, indices = similarity.topk(k, dim=0)
  return indices.detach().cpu().numpy().ravel(), values.detach().cpu().numpy().ravel()



In [82]:
k = 3
search_query = "Where they talk about Matrix"
indices, scores = search(search_query, k=k) # should already be sorted by scores descending


In [83]:
indices, scores

(array([37, 34, 44]),
 array([0.8177787 , 0.81579757, 0.8144379 ], dtype=float32))

In [84]:
matching_segments = []
for ind in indices:
  matching_segments.append(emb_index[ind])


In [85]:
matching_segments

[{'start': 2165.64,
  'end': 2230.1600000000003,
  'text': "You know, you gotta be in the Matrix to stop bullets.Come on now.Right.Exactly.You know?Yeah.So, anyhow, so the hopefuls who were waiting in line to see the oracles ...Oh, the potentials.Yeah, the potentials.Yeah.Yeah.Yeah.So, okay, fine.All right.Fine.It's a nice visual.Yeah.It's kind of cool.Yeah.Also, the oracle is establishing that Trinity's in love with him.Right.And he's oblivious to that.Okay.I was kind of oblivious to that too.I'm just, you know, guys are just generally stupid, you know, about these things.So, and she says, you're not too smart either.Not too bright.Not too bright.She's in you.Who?Not too bright either.Yeah, it's great.It's really cool.Yeah.Yeah.Because given the choice, you'll have Morpheus in one hand and Trinity in the other.And you know, so I think it was brilliantly acted by Keanu Reeves where he's in the carand they're driving and he says, good sushi.I used to have noodles."},
 {'start': 1998.16,

In [86]:
# sort results by start time asceding
matching_segments = sorted(matching_segments, key=lambda x: x['start'])

In [87]:
def clip_video(video_path, start_time, end_time, output_path, filename, file_ext="mp4"):
  # !ffmpeg -hide_banner -i "$video_path" -ss "$start_time" -to "$end_time" -c copy "$output_path"/"$filename"."$file_ext"
  !ffmpeg -y -hide_banner -i "$video_path" -ss "$start_time" -to "$end_time" -vf "scale=iw*0.5:ih*0.5" -c:v libx264 -crf 23 -preset fast -c:a aac -b:a 128k "$output_path"/"$filename"."$file_ext"

In [88]:
mp4_path

'sample_data/The Science of the Matrix with Laurence Fishburne.mp4'

In [89]:
os.makedirs(f"sample_data/clips/{search_query}", exist_ok=True)
for i, segment in enumerate(matching_segments):
  clip_start = segment["start"]
  clip_end = segment["end"]
  clip_video(mp4_path, clip_start, clip_end, f"sample_data/clips/{search_query}", f"{int(clip_start)}_clip_{i}")


Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'sample_data/The Science of the Matrix with Laurence Fishburne.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: isommp42
    encoder         : Google
  Duration: 01:00:48.53, start: 0.000000, bitrate: 484 kb/s
  Stream #0:0(und): Video: h264 (Main) (avc1 / 0x31637661), yuv420p(tv, bt709), 640x360 [SAR 1:1 DAR 16:9], 386 kb/s, 23.98 fps, 23.98 tbr, 24k tbn, 47.95 tbc (default)
    Metadata:
      handler_name    : ISO Media file produced by Google Inc.
      vendor_id       : [0][0][0][0]
  Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 96 kb/s (default)
    Metadata:
      handler_name    : ISO Media file produced by Google Inc.
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x5a924a4e83c0] using SAR=1/1
[libx264 @ 0x

In [90]:
# list all clips for query

clips_dir = os.listdir(f"sample_data/clips/{search_query}")
clips = [f for f in clips_dir if f.endswith(".mp4")]
clips = sorted(clips, key=lambda x: float(x.split("_")[0]))

clip_files = []
for clip in clips:
  clip_files.append(f"sample_data/clips/{search_query}/{clip}")
print(clip_files)

['sample_data/clips/Where they talk about Matrix/1998_clip_0.mp4', 'sample_data/clips/Where they talk about Matrix/2165_clip_1.mp4', 'sample_data/clips/Where they talk about Matrix/2600_clip_2.mp4']


In [91]:
# display clips for the search
from IPython.display import HTML
from base64 import b64encode

mp4 = open(clip_files[0],'rb').read()

data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)